# **Spark SQL Tutorial**


Функции, которые будут рассмотрены.

Transformations:

* `select()`, `filter()`, `distinct()`, `dropDuplicates()`, `orderBy()`, `groupBy()`

Actions:
* `first()`, `take()`, `count()`, `collect()`, `show()`

Кэширование:
* `cache()`, `unpersist()`

Документация [Spark's PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)

##   [Apache Spark](https://spark.apache.org/) с модулем [PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)**

### Spark Context

В Spark, основная коммуникация происходит между driver и executors. Когда мы используем Spark, то driver обращается на Java Virtual Machine (JVM), ожидая запуска выполнения, до тех пор, пока не будет выполнен action. После чего запускается процесс выполнения на executors.

Для запуска Spark SQL, нужно запустить `SQLContext`.  Каждая сессия работает с одним [SparkContext]. После запуска `SparkContext` мы инициируем работу с любым другим контекстом `SQLContext` и др.

Driver общается с Spark через объекты SparkContext, которые являются "точкой входа" для построения задач. Пример,Spark SQL контекст (`sqlContext`) в DataBricks содержит Spark DataFrame и SQL функционал (на кластере, вы можете использовать SparkSession / SQLContext / HiveContex) 

In [ ]:
# просмотреть тип контекста (на DataBricks)
type(sqlContext)

In [ ]:
# просмотреть тип контекста (на Cluster (локальный))

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

type(spark)

**Note:** `HiveContext` означает, что вы будет работаете с обхектами, которые поддерживают Hive ([Spark Programming Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html#starting-point-sqlcontext)), если вы не имеете Hive на кластере, то у вас нет мета-хранилища данных, которые содержит Hive. `HiveContext` - это надстройка над `SQLContext`, которая содержит "фичи" для работы с HiveQL, Hive UDFs и Hive таблицами. 

### SparkContext attributes

Для просмотра атрибутов контекста, вы можете воспользоваться [dir()](https://docs.python.org/2/library/functions.html?highlight=dir#dir) функцией

In [19]:
# список sqlContext
dir(sqlContext)

### Help

Помощь по объекту. Функция [help()](https://docs.python.org/2/library/functions.html?highlight=help#help) 

In [21]:
help(sqlContext)

## DataFrames

### Основы работы с DataFrames

Следующая структура, после RDD, это [DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame).

Методы работы с Spark DF похожи на Pandas DF, будет легко понять и начать использовать.

Принцип работы, как с RDD:
- DF  - это не изменяемый объект
- Сделайте трансформер (ленивые вычисления)
- Для запуска используйте Экшен 

Сделаем следущие шагиs:
* Создадим Python объект из 10,000 int
* Создадим Spark DataFram
* Применим `map`
* Применим `collect`
* Применим `count`
* Применим `filter` и `collect`
* Повторим Lambda
* Исследуем оценнку производительности и дебаггинг

DataFrame - это объект из `Row`; каждая `Row` - это набор из namedtuple объектов. Можно представлять себе это, как таблицу.

### Python collection

Используем библиотеку [fake-factory](https://pypi.python.org/pypi/fake-factory/0.5.3) to create a collection of fake person records.

In [6]:
from faker import Factory
fake = Factory.create()
fake.seed(4321)


In [30]:
# создадим объект из last_name, first_name, ssn, job, age

from pyspark.sql import Row
def fake_entry():
    name = fake.name().split()
    return (name[1], name[0], fake.ssn(), fake.job(), abs(2016 - fake.date_time().year) + 1)

In [31]:
# вспомогательная функция для вызова других функций в повторе
def repeat(times, func, *args, **kwargs):
    for _ in range(times):
        yield func(*args, **kwargs)

In [32]:
data = list(repeat(10000, fake_entry))

In [34]:
data[0]

In [36]:
len(data)

### Данные распределены 


<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3b.png" style="width: 900px; float: right; margin: 5px"/>


`createDataFrame()`

In [39]:
help(sqlContext.createDataFrame)

In [40]:
# DF с колонками
dataDF = sqlContext.createDataFrame(data, ('last_name', 'first_name', 'ssn', 'occupation', 'age'))

In [42]:
print('type of dataDF: {0}'.format(type(dataDF)))

In [44]:
# обзор схемы
dataDF.printSchema()

Используем метод `registerDataFrameAsTable()` для регистрации данного DF, как таблицы.
Это позволит обращаться к данным, через SQL запросы

In [46]:
sqlContext.registerDataFrameAsTable(dataDF, 'dataframe')

In [48]:
help(dataDF)

Как распределенны данные

In [50]:
# DF.type.Method <- обратите внимание на последовательность! Где вы ещё её видели? 
dataDF.rdd.getNumPartitions()

###### DataFrames и queries

Когда мы используем DataFrames или Spark SQL, мы используем "под капотом" _query plan_. Каждая трансформация, до экшена, записывается в него. После экшена происходит следущее:

1. Spark Catalyst оптимизирует план запроса (без оптимизации план называется _unoptimized logical query plan_). Оптимизация включает в себя пересмотр последовательностей `filter()` и расчет эффективности операций. Часто происходит переконвертация `Decimal` значений в более эффективный long integer. На выходе оптимизатора -  _optimized logical plan_.

2. Оптимизированный план строит _physical_ план. Это план для элементов RDD.

3. Catalyst делаетрасчет _cost optimization_.

4. DataFrame выполняется, как RDD на основе плана выполнения.

План можно получить методом `explain()`

Подробно в документации [Deep Dive into Spark SQL's Catalyst Optimizer](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)

In [52]:
newDF = dataDF.distinct().select('*')
newDF.explain(True)

### SELECT


In [54]:
# Трансформируем dataDF методом select.
# Метод Select позволяет не только делать выбор, но и трансформировать колонки (age колонка - 1)
subDF = dataDF.select('last_name', 'first_name', 'ssn', 'occupation', (dataDF.age - 1).alias('age'))

In [56]:
subDF.explain(True)

### COLLECT

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3d.png" style="height:700px;float:right"/>


In [58]:
results = subDF.collect()
print(results)

In [60]:
subDF.show()

In [62]:
subDF.show(n=30, truncate=False)

In [64]:
display(subDF)

### COUNT

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3e.png" style="height:700px;float:right"/>


In [66]:
print(dataDF.count())
print(subDF.count())

### FILTER и COLLECT

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3f.png" style="height:700px;float:right"/>



In [68]:
filteredDF = subDF.filter(subDF.age < 10)
filteredDF.show(truncate=False)
filteredDF.count()

## Python Lambda и User Defined Functions

Мы можем создавать свои собственные функции. Это необходимо, когда нет другого выхода, но обработать DF надо.

Рассмотрим пример: не будем использовать метод `filter()`, а сделаем самостоятельную функцию с `lambda()` и из неё создадим Spark User Defined Function (UDF)

In [71]:
from pyspark.sql.types import BooleanType
less_ten = udf(lambda s: s < 10, BooleanType())
lambdaDF = subDF.filter(less_ten(subDF.age))
lambdaDF.show()
lambdaDF.count()

In [72]:
even = udf(lambda s: s % 2 == 0, BooleanType())
evenDF = lambdaDF.filter(even(lambdaDF.age))
evenDF.show()
evenDF.count()

## Actions (выбор данных)


* [first()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.first)
* [take()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.take)


In [74]:
print("first: {0}\n".format(filteredDF.first()))

print("Four of them: {0}\n".format(filteredDF.take(4)))

In [76]:
# более приятный вид
display(filteredDF.take(4))

## Трансформаторы DF

### orderBy

[`orderBy()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) 

* Pandas-style notation: `filteredDF.age`
* Subscript notation: `filteredDF['age']`


Пример применения:

```
dataDF.orderBy(dataDF['age'])  # sort by age in ascending order; returns a new DataFrame
dataDF.orderBy(dataDF.last_name.desc()) # sort by last name in descending order
```

In [79]:
# получить 5 самых старых людей в DF
display(dataDF.orderBy(dataDF.age.desc()).take(5))

Можно использовать `desc()` и `asc()` методы по колонке `orderBy('age'.desc())` 

In [81]:
display(dataDF.orderBy('age').take(5))

### distinct и dropDuplicates

[`distinct()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) 


!Note:  `fake-factory` создает мало дублей (или вообще не создает)

In [83]:
print(dataDF.count())
print(dataDF.distinct().count())

In [85]:
tempDF = sqlContext.createDataFrame([("Vasya", 1), ("Vasya", 1), ("Anna", 15), ("Anna", 12), ("Roman", 5)], ('name', 'score'))

In [86]:
tempDF.show()

In [87]:
tempDF.distinct().show()

("Vasya", 1) будет удалено, но строчки с именем "Anna" останутся, так как мы применяем ко всей строке метод

[`dropDuplicates()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates) как `distinct()`, но позволяет указать конкретные колонки для выполнения

In [90]:
print(dataDF.count())
print(dataDF.dropDuplicates(['first_name', 'last_name']).count())

### drop

[`drop()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop)

In [92]:
dataDF.drop('occupation').drop('age').show()

### groupBy

[`groupBy()`]((http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) 

`groupBy()` не возвращает DataFrame, она возвращает объект [GroupedData](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData), которые содержит функции, после выполнения их вы получите новый DF:

* [count()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.count)
* [sum()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.sum)
* [max()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.max)
* [avg()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.avg)


In [94]:
dataDF.groupBy('occupation').count().show(truncate=False)

In [95]:
dataDF.groupBy().avg('age').show(truncate=False)

In [97]:
print("Maximum age: {0}".format(dataDF.groupBy().max('age').first()[0]))
print("Minimum age: {0}".format(dataDF.groupBy().min('age').first()[0]))

### sample

[`sample()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample) может выполнятся с аргументом `withReplacement`, что означает "фиксировать или нет значения в сэмле".  `withReplacement=True` будут возвращаться одни и те же данные

`fraction` это процент от общего количества данных (`0.20`)

In [99]:
sampledDF = dataDF.sample(withReplacement=False, fraction=0.10)
print(sampledDF.count())
sampledDF.show()

In [100]:
print(dataDF.sample(withReplacement=False, fraction=0.05).count())

## Caching DataFrame

In [103]:
# Cache DataFrames
filteredDF.cache()
# action
print(filteredDF.count())
# проверка на кэширование
print(filteredDF.is_cached)

### Unpersist 

`unpersist()` удаляет данные из кэша Spark

так же можно применить метод [persist()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.persist) 

In [105]:
# разкэшировать объект
filteredDF.unpersist()
# проверка
print(filteredDF.is_cached)

## Debugging Spark

Проблема с поиском ошибок в том, что Spark код не выполняется сразу. Разберем пример, как искать ошибки.

Создадим функцию, которая не правильно определяет значение в функции

Используем `filter()` метод для фильтрации элементов. Ошибок не появится.

`filter()` метод не исполняемый до тех пор, пока не будет action. Используем метод `count()` для запуска вычислений

In [109]:
def brokenTen(value):
    """Не правильная функция

    Note:
        В `if` условии используется `val`, а не `value`.

    Args:
        value (int)

    Returns:
        bool

    Raises:
        NameError
    """
    if (val < 10):
        return True
    else:
        return False

btUDF = udf(brokenTen)
brokenDF = subDF.filter(btUDF(subDF.age) == True)

In [110]:
# выполним код
brokenDF.count()

### Поиск ошибок

?????? КАК ИСКАТЬ ????????

`NameError: global name 'val' is not defined`


###  Способы поиска


Способ, который стоит использовать во время учебы:
```
    df2 = df1.transformation1()
    df2.action1()
    df3 = df2.transformation2()
    df3.action2()
```

Способ более прокаченный: 
`df.transformation1().transformation2().action()`


СТОП! Это же просто запуск выполнения! А где же дебаг? (Ждем занятия 8 =) )

In [113]:
# пример более лучшего исполнения
myUDF = udf(lambda v: v < 10)
subDF.filter(myUDF(subDF.age) == True)

### Пару слов о "code style"

In [115]:
from pyspark.sql.functions import *
(dataDF
 .filter(dataDF.age > 20)
 .select(concat(dataDF.first_name, lit(' '), dataDF.last_name), dataDF.occupation)
 .show(truncate=False)
 )

In [7]:
# DataFrame
# Select
# Filters
# Transformators
# Select
# Filters
# Select
# Action